<a href="https://colab.research.google.com/github/agron-gojcaj/IntrusionDetection/blob/main/Intrusion_Detection_NSLKDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

# Step 1: Load Data
train_df = pd.read_csv('kdd_train.csv')
test_df = pd.read_csv('kdd_test.csv')

# Step 2: Encode categorical features
categorical_cols = ['protocol_type', 'service', 'flag']
for col in categorical_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])

# Step 3: Convert labels to binary (normal = 0, attack = 1)
train_df['labels'] = train_df['labels'].apply(lambda x: 0 if x == 'normal' else 1)
test_df['labels'] = test_df['labels'].apply(lambda x: 0 if x == 'normal' else 1)

# Step 4: Split features and labels
X_train = train_df.drop('labels', axis=1).values.astype(np.float32)
y_train = train_df['labels'].values.astype(np.float32)

X_test = test_df.drop('labels', axis=1).values.astype(np.float32)
y_test = test_df['labels'].values.astype(np.float32)

# Step 5: Normalize features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 6: Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train).unsqueeze(1)
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test).unsqueeze(1)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)

# Step 7: Define the neural network
class IntrusionDetectionNet(nn.Module):
    def __init__(self, input_dim):
        super(IntrusionDetectionNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Step 8: Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = IntrusionDetectionNet(X_train.shape[1]).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(10):
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}")

# Step 9: Evaluate
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor.to(device)).cpu().numpy()
y_pred_binary = (y_pred > 0.5).astype(int)

# Step 10: Print results
print(f"Accuracy: {accuracy_score(y_test, y_pred_binary):.4f}")
print(classification_report(y_test, y_pred_binary, target_names=["Normal", "Attack"]))

Epoch 1: Loss = 126.4034
Epoch 2: Loss = 62.5889
Epoch 3: Loss = 45.2675
Epoch 4: Loss = 36.9164
Epoch 5: Loss = 31.0689
Epoch 6: Loss = 27.1144
Epoch 7: Loss = 24.4022
Epoch 8: Loss = 22.9856
Epoch 9: Loss = 21.8926
Epoch 10: Loss = 21.0980
Accuracy: 0.9278
              precision    recall  f1-score   support

      Normal       0.88      0.99      0.93     11245
      Attack       0.99      0.87      0.92     11299

    accuracy                           0.93     22544
   macro avg       0.93      0.93      0.93     22544
weighted avg       0.93      0.93      0.93     22544

